In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')


from pathlib import Path
import os, re, shutil, sys
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [ ]:
TRAIN_FASTA = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
TRAIN_TERMS = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv"
TRAIN_TAXONOMY = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv"

TEST_FASTA  = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta"
TEST_TAXON  = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv"

IA_TSV      = "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv"  # whitelist of GO terms

OUT_BASE   = "/kaggle/working/submission_baseline.tsv"
OUT_KAGGLE = "/kaggle/working/submission.tsv"

In [ ]:
# ==========================================================
# CAFA-6: XGBoost (OvR) Baseline with Kaggle GPU
# - Features: TF-IDF (3-mer) + optional Taxonomy one-hot
# - IA.tsv whitelist for allowed GO terms
# - GPU if available (gpu_hist), else CPU fallback (hist)
# - Generous emission (avoid 0.000 submissions)
# - Auto-writes /kaggle/working/submission.tsv
# ==========================================================


# -------------------- PATHS --------------------


# -------------------- PARAMS --------------------
# Features
KMER_K        = 3
USE_TAXONOMY  = True  # set False to skip taxonomy features

# Label space control (important for runtime)
MIN_COUNT     = 2       # keep GO terms appearing >= 2 times
MAX_TERMS     = 1200    # cap number of GO classes (raise if your session allows)

# Emission control (be generous; evaluator picks the threshold)
MIN_PROB      = 1e-6
TOP_K         = 200
CAP_PER_PROT  = 1500

# XGBoost params
XGB_ROUNDS                = 200          # num_boost_round (increase if time allows)
EARLY_STOPPING_ROUNDS     = 20
MAX_DEPTH                 = 6
ETA                       = 0.1
SUBSAMPLE                 = 0.8
COLSAMPLE_BYTREE          = 0.8
LAMBDA_L2                 = 1.0

# Training loop controls
MIN_POSITIVES_PER_CLASS   = 5            # skip classes with too few positives
VAL_SIZE                  = 0.1          # class-wise validation split for early stopping
RANDOM_STATE              = 42

# Speed sanity-check (set to False for real submission)
DEBUG_SMALL   = False
DEBUG_N_TRAIN = 1200     # #labelled proteins to use (train)
DEBUG_N_TEST  = 600      # #test proteins to predict

# -------------------- UTILITIES --------------------
def read_fasta_ids_and_seqs(path):
    ids, seqs, cur_id, cur_seq = [], [], None, []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith(">"):
                if cur_id is not None:
                    ids.append(cur_id); seqs.append("".join(cur_seq))
                header = line[1:].strip()
                cur_id = header.split("|")[1].split()[0] if "|" in header else header.split()[0]
                cur_seq = []
            else:
                cur_seq.append(line.strip())
        if cur_id is not None:
            ids.append(cur_id); seqs.append("".join(cur_seq))
    return ids, seqs

def load_train_terms(path):
    df0 = pd.read_csv(path, sep="\t", dtype=str)
    cols = [c.lower() for c in df0.columns]; df0.columns = cols
    if "entryid" in cols and "term" in cols:
        df = df0.rename(columns={"entryid":"protein_id","term":"go_id"})[["protein_id","go_id"]]
    else:
        df = df0.iloc[:, :2].copy()
        df.columns = ["protein_id","go_id"]
    return df.dropna().drop_duplicates()

def load_taxonomy(path):
    df = pd.read_csv(path, sep="\t", header=None, dtype=str)
    if df.shape[1] < 2:
        raise ValueError("taxonomy TSV must have at least two columns.")
    df = df.iloc[:, :2].copy()
    df.columns = ["protein_id","taxon"]
    return dict(zip(df["protein_id"], df["taxon"]))

def build_label_sets(df_terms):
    mp = defaultdict(set)
    for p,g in df_terms[["protein_id","go_id"]].itertuples(index=False):
        mp[p].add(g)
    return mp

def filter_terms_by_freq(p2t, min_count=2, max_terms=None):
    c = Counter()
    for ts in p2t.values(): c.update(ts)
    items = [(g,n) for g,n in c.items() if n >= min_count]
    items.sort(key=lambda x: x[1], reverse=True)
    if max_terms: items = items[:max_terms]
    return set(g for g,_ in items)

def load_allowed_go_terms(ia_path):
    df = pd.read_csv(ia_path, sep="\t", dtype=str)
    # Try to find a column that contains GO ids
    go_cols = [c for c in df.columns if "go" in c.lower() or "term" in c.lower()]
    if go_cols:
        vals = df[go_cols[0]].dropna().astype(str).tolist()
        return {v for v in vals if v.startswith("GO:")}
    # Fall back: scan all columns
    s = set()
    for col in df.columns:
        s |= set(df[col].dropna().astype(str))
    return {x for x in s if x.startswith("GO:")}

def round_sig(x: float, sig=3) -> str:
    if x <= 0: return None
    s = f"{x:.{sig}g}"
    return s if float(s) > 0 else "0.001"

def make_submission_rows(pids, gids, proba, min_prob=1e-6, top_k=200, cap=1500):
    rows = []
    for i, pid in enumerate(pids):
        pr = proba[i]
        idx = np.where(pr >= min_prob)[0]
        if top_k: idx = np.unique(np.concatenate([idx, np.argsort(-pr)[:top_k]]))
        idx = idx[np.argsort(-pr[idx])]
        count = 0
        for j in idx:
            if count >= cap: break
            sc = round_sig(pr[j], 3)
            if sc: rows.append((pid, gids[j], sc)); count += 1
    return rows

def get_xgb_device_params():
    # Use GPU if available; else CPU
    try:
        import subprocess
        has_gpu = False
        try:
            out = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
            has_gpu = (out.returncode == 0)
        except Exception:
            has_gpu = False
        if has_gpu:
            return dict(tree_method="gpu_hist", predictor="gpu_predictor")
    except Exception:
        pass
    return dict(tree_method="hist", predictor="auto")

def train_xgb_gpu_binary(X, y, params, rounds=200, early_stopping=20, val_size=0.1, seed=42):
    # Ensure we have both classes for validation
    stratify = y if y.sum() > 0 and y.sum() < len(y) else None
    X_tr, X_va, y_tr, y_va = train_test_split(
        X, y, test_size=val_size, random_state=seed, stratify=stratify
    )
    dtr = xgb.DMatrix(X_tr, label=y_tr)
    dva = xgb.DMatrix(X_va, label=y_va)
    evals = [(dtr, "train"), (dva, "valid")]
    bst = xgb.train(params, dtr, num_boost_round=rounds, evals=evals,
                    early_stopping_rounds=early_stopping, verbose_eval=False)
    return bst

# -------------------- GPU INFO --------------------
print("XGBoost version:", xgb.__version__)
dev_params = get_xgb_device_params()
print("XGB device params:", dev_params)

# -------------------- LOAD DATA --------------------
print("Loading training FASTA...")
train_ids, train_seqs = read_fasta_ids_and_seqs(TRAIN_FASTA)
print("Loading training terms...")
terms_df = load_train_terms(TRAIN_TERMS)
prot2terms_all = build_label_sets(terms_df)

# Keep only proteins with labels and present in FASTA
X_text, y_terms, used_ids = [], [], []
for pid, seq in zip(train_ids, train_seqs):
    t = prot2terms_all.get(pid)
    if t:
        used_ids.append(pid); X_text.append(seq); y_terms.append(sorted(t))
print("Training instances used:", len(used_ids))

if DEBUG_SMALL:
    n_keep = min(DEBUG_N_TRAIN, len(used_ids))
    used_ids = used_ids[:n_keep]
    X_text = X_text[:n_keep]
    y_terms = y_terms[:n_keep]

# Term filtering (frequency) and whitelist (IA.tsv)
kept_freq = filter_terms_by_freq({pid:set(ts) for pid,ts in zip(used_ids, y_terms)},
                                 min_count=MIN_COUNT, max_terms=MAX_TERMS)
allowed_terms = load_allowed_go_terms(IA_TSV)
print("Kept by freq:", len(kept_freq), "| Allowed (IA):", len(allowed_terms))
kept_terms = kept_freq & allowed_terms
print("Kept after whitelist:", len(kept_terms))
y_filtered = [[t for t in ts if t in kept_terms] for ts in y_terms]

# -------------------- FEATURES --------------------
print("Vectorizing sequences with TF-IDF(3-mer)...")
vect = TfidfVectorizer(analyzer="char", ngram_range=(KMER_K, KMER_K), lowercase=False)
X_seq = vect.fit_transform(X_text)

parts_train = [X_seq]

# Optional: taxonomy features



In [ ]:
tax_info = None
if USE_TAXONOMY:
    try:
        train_tax = load_taxonomy(TRAIN_TAXONOMY)
        test_tax  = load_taxonomy(TEST_TAXON)
        dv = DictVectorizer(sparse=True)
        X_tax_train = dv.fit_transform([{"taxon": train_tax.get(pid, "NA")} for pid in used_ids])
        parts_train.append(X_tax_train)
        tax_info = (test_tax, dv)
        print("Taxonomy dims:", X_tax_train.shape)
    except Exception as e:
        print("Taxonomy disabled:", e)
        tax_info = None

X_train = parts_train[0] if len(parts_train) == 1 else sparse.hstack(parts_train).tocsr()
print("X_train shape:", X_train.shape)

# Labels (multi-label -> OvR)
mlb = MultiLabelBinarizer()
Y_train = mlb.fit_transform(y_filtered)
classes = list(mlb.classes_)
print("Final class count:", len(classes))
if Y_train.shape[1] == 0:
    raise RuntimeError("No GO classes after filtering. Relax MIN_COUNT and/or raise MAX_TERMS.")

# -------------------- TRAIN OvR XGBoost --------------------
xgb_base_params = dict(
    objective="binary:logistic",
    eval_metric="auc",
    max_depth=MAX_DEPTH,
    eta=ETA,
    subsample=SUBSAMPLE,
    colsample_bytree=COLSAMPLE_BYTREE,
    reg_lambda=LAMBDA_L2,
    **dev_params,
)

models = []
kept_class_indices = []  # indices of trained classes (some may be skipped)
skipped = 0

for k in range(Y_train.shape[1]):
    y = Y_train[:, k].astype(np.float32)
    pos = int(y.sum())
    if pos < MIN_POSITIVES_PER_CLASS:
        models.append(None)
        skipped += 1
        continue
    bst = train_xgb_gpu_binary(
        X_train, y, xgb_base_params,
        rounds=XGB_ROUNDS,
        early_stopping=EARLY_STOPPING_ROUNDS,
        val_size=VAL_SIZE,
        seed=RANDOM_STATE
    )
    models.append(bst)
    kept_class_indices.append(k)

print(f"Trained {len(kept_class_indices)} classes; skipped {skipped} with < {MIN_POSITIVES_PER_CLASS} positives.")

# -------------------- BUILD TEST FEATURES --------------------
print("Loading test FASTA...")
test_ids_full, test_seqs_full = read_fasta_ids_and_seqs(TEST_FASTA)
if DEBUG_SMALL:
    test_ids = test_ids_full[:DEBUG_N_TEST]
    test_seqs = test_seqs_full[:DEBUG_N_TEST]
else:
    test_ids, test_seqs = test_ids_full, test_seqs_full

X_test_seq = vect.transform(test_seqs)
parts_test = [X_test_seq]

if tax_info:
    test_tax, dv = tax_info
    X_tax_test = dv.transform([{"taxon": test_tax.get(pid, "NA")} for pid in test_ids])
    parts_test.append(X_tax_test)

X_test = parts_test[0] if len(parts_test) == 1 else sparse.hstack(parts_test).tocsr()
print("X_test shape:", X_test.shape)

# -------------------- PREDICT --------------------
print("Predicting probabilities with XGBoost (GPU if available)...")
dtest = xgb.DMatrix(X_test)
proba = np.zeros((X_test.shape[0], Y_train.shape[1]), dtype=np.float32)

for k, bst in enumerate(models):
    if bst is None:
        continue
    # use best_ntree_limit if available
    best_ntree = getattr(bst, "best_ntree_limit", 0)
    if best_ntree and best_ntree > 0:
        proba[:, k] = bst.predict(dtest, ntree_limit=best_ntree)
    else:
        proba[:, k] = bst.predict(dtest)

# -------------------- WRITE SUBMISSION --------------------


In [ ]:
print("Formatting submission rows...")
rows = make_submission_rows(
    pids=test_ids,
    gids=classes,
    proba=proba,
    min_prob=MIN_PROB,
    top_k=TOP_K,
    cap=CAP_PER_PROT
)

print(f"Writing {len(rows)} rows -> {OUT_BASE}")
with open(OUT_BASE, "w", encoding="utf-8") as f:
    for pid, go, sc in rows:
        f.write(f"{pid}\t{go}\t{sc}\n")

# Auto-copy for Kaggle Submission
shutil.copy(OUT_BASE, OUT_KAGGLE)
print("✅ submission.tsv ready at:", OUT_KAGGLE)

# Preview
!head -n 5 /kaggle/working/submission.tsv || true
!wc -l /kaggle/working/submission.tsv || true